In [1]:
import tensorflow as tf

In [2]:
# TODO: update it to load from TFHub later
loaded = tf.saved_model.load("../src/saved-model")

In [3]:
list(loaded.signatures.keys())

['infer', 'train']

In [4]:
model = loaded.signatures["train"]

@tf.function(jit_compile=True)
def forward(batch):
    return model(batch)

In [5]:
batch = tf.random.uniform(shape=(2, 246000))
forward(batch)

{'output_0': <tf.Tensor: shape=(2, 768, 768), dtype=float32, numpy=
 array([[[ 0.04981414, -0.11246528,  0.22671068, ...,  0.4736007 ,
           0.0594595 , -0.16712372],
         [-0.05188152, -0.14415626,  0.21715117, ...,  0.4496262 ,
           0.19341631, -0.128913  ],
         [ 0.21963328, -0.04843174,  0.10294376, ...,  0.36572003,
           0.15869941,  0.03712311],
         ...,
         [ 0.033603  , -0.12690043,  0.10068704, ...,  0.47456473,
           0.06561999, -0.08360367],
         [ 0.14492346, -0.14835656,  0.09673587, ...,  0.53533024,
           0.25344235, -0.00547679],
         [ 0.04792857, -0.21567795,  0.177122  , ...,  0.3946013 ,
           0.12262318, -0.16006427]],
 
        [[ 0.08920608, -0.00930583,  0.224908  , ...,  0.2948055 ,
           0.02860889, -0.05696075],
         [-0.031082  , -0.05461206,  0.19720137, ...,  0.28247502,
           0.0928454 , -0.06703105],
         [ 0.22385038, -0.17530268,  0.2455038 , ...,  0.3629407 ,
           0.228

In [ ]:
# TODO: add how to train?